In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 12.0 MB/s eta 0:00:00


In [ ]:
!pip install mysql-connector-python
import mysql.connector
import pandas as pd
import numpy as np
import os

# Load MySQL credentials from environment variables
DB_HOST = os.getenv("DB_HOST", "Host IP")
DB_USER = os.getenv("DB_USER", "user name")
DB_PASSWORD = os.getenv("DB_PASSWORD", "password")
DB_NAME = os.getenv("DB_NAME", "DB name")

# MySQL connection
conn = mysql.connector.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME
)
cursor = conn.cursor()



# Define your stock universe
stock_universe = []  # Add your own stocks



# Query to fetch stock price data
query = """
    SELECT record_date, stockname, close
    FROM quandl
    WHERE record_date >= DATE_SUB(CURDATE(), INTERVAL 13 MONTH)
    ORDER BY stockname, record_date;
"""
df = pd.read_sql(query, conn)

# Convert record_date to datetime
df['record_date'] = pd.to_datetime(df['record_date'])

# Filter by stock universe
df = df[df['stockname'].isin(stock_universe)]

# Compute Log Returns
df['log_return'] = np.log(df['close'] / df.groupby('stockname')['close'].shift(1))

# Compute Rolling Standard Deviations
df['rolling_std12'] = df.groupby('stockname')['log_return'].rolling(window=252, min_periods=50).std().reset_index(0, drop=True)
df['rolling_std6'] = df.groupby('stockname')['log_return'].rolling(window=126, min_periods=50).std().reset_index(0, drop=True)

### --- Calculate Momentum Returns (12M & 6M) Based on Month-End Prices --- ###

# Step 1: Extract month-end close prices
df_monthly = df.groupby(['stockname', df['record_date'].dt.to_period('M')])['close'].last().reset_index()

# Step 2: Convert period to actual month-end date
df_monthly['record_date'] = df_monthly['record_date'].dt.to_timestamp()

# Step 3: Compute 12-month & 6-month percentage changes
df_monthly['12M_return'] = df_monthly.groupby('stockname')['close'].pct_change(periods=12)
df_monthly['6M_return'] = df_monthly.groupby('stockname')['close'].pct_change(periods=6)

# Step 4: Merge monthly returns back into the main DataFrame
df = pd.merge(df, df_monthly[['stockname', 'record_date', '12M_return', '6M_return']],
              on=['stockname', 'record_date'], how='left')

### --- Compute Momentum Ratios --- ###
df['MR12'] = df['12M_return'] / (df['rolling_std12'] + 1e-9)  # Avoid division by zero
df['MR6'] = df['6M_return'] / (df['rolling_std6'] + 1e-9)

# Handle cases where we get NaN or infinite values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Compute Mean & Std Dev of Momentum Ratios for Universe
MR12_mean = df['MR12'].mean(skipna=True)
MR12_std = df['MR12'].std(skipna=True)
MR6_mean = df['MR6'].mean(skipna=True)
MR6_std = df['MR6'].std(skipna=True)

# Compute Z Scores
df['Z12'] = (df['MR12'] - MR12_mean) / (MR12_std + 1e-9)  # Prevent division by zero
df['Z6'] = (df['MR6'] - MR6_mean) / (MR6_std + 1e-9)

# Compute Weighted Average Z Score
df['Weighted_Z'] = 0.5 * df['Z12'] + 0.5 * df['Z6']

# Compute Normalized Momentum Score
df['Norm_Momentum_Score'] = np.where(
    df['Weighted_Z'] >= 0,
    (1 + df['Weighted_Z']),
    (1 - df['Weighted_Z'])**-1
)

# Select Top 30 Stocks based on Normalized Momentum Score
top_stocks = df.sort_values(by='Norm_Momentum_Score', ascending=False).head(20)

# Print the top stocks
print(top_stocks[['stockname', 'Norm_Momentum_Score']])

# Close DB connection
cursor.close()
conn.close()
